In [1]:
import pickle
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from statsmodels.formula.api import ols
import numpy as np
import statsmodels.api as sm
import researchpy as rp
import os
import re

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [3]:
root_dir = os.getcwd() + '\\pickles\\' # put all the pickle files to this directory
val_acc = []
avg_val_acc=[]
val_loss=[]
label_error=[]
image_noise=[]
labeler=[]
repetition=[]
for file in os.listdir(root_dir):
    file_name = root_dir + file
    filein = open(file_name, "rb")
    info = pickle.load(filein, encoding="utf8")
    val_acc.extend(info[14][0])
    s_bnn = 0
    for i in range(0, 10):
        s_bnn = s_bnn + val_acc[-5 * i - 1]
    average_valacc = s_bnn / 10
    avg_val_acc.append(average_valacc)
    val_loss.extend(info[16][0])
    label_error.append(re.findall('\d+', file)[9])
    image_noise.append(re.findall('\d+', file)[10])
    labeler.append(re.search(r'True|False', file).group())
    repetition.append(re.findall('\d+', file)[11])

    filein.close()

df = pd.DataFrame({'label_error':label_error,
                            'image_noise':image_noise,
                            'labeler':labeler, 
                            'val_acc':avg_val_acc})

In [4]:
df

,label_error,image_noise,labeler,val_acc
0,0,0,False,0.93203
1,0,40,False,0.92339
2,40,0,False,0.88229
3,40,40,False,0.85073
4,0,0,False,0.93282
5,0,0,False,0.93007
6,0,0,True,0.93489
7,0,0,True,0.92385
8,0,0,True,0.92386
9,0,40,False,0.92187


In [5]:
rp.summary_cont(df.groupby(list(df.columns[:-1].values)))[df.columns[-1]]

N    Mean      SD      SE  95% Conf.  \
label_error image_noise labeler                                         
0           0           False    3  0.9316  0.0014  0.0008     0.9281   
                        True     3  0.9275  0.0064  0.0037     0.9117   
            40          False    3  0.9163  0.0109  0.0063     0.8892   
                        True     3  0.8929  0.0120  0.0069     0.8632   
40          0           False    3  0.8855  0.0043  0.0025     0.8749   
                        True     3  0.9227  0.0068  0.0039     0.9059   
            40          False    3  0.8618  0.0099  0.0057     0.8372   
                        True     3  0.8851  0.0186  0.0107     0.8390   

                                 Interval  
label_error image_noise labeler            
0           0           False      0.9352  
                        True       0.9434  
            40          False      0.9434  
                        True       0.9227  
40          0           False      0.8961  
                        True       0.9396  
            40          False      0.8864  
                        True       0.9312

In [6]:
model_string = "%s ~ "%df.columns[-1]+"".join(["C(%s)*"%i if i != df.columns[-1] else '' for i in df.columns])[:-1]

In [7]:
model = ols(model_string, data=df).fit()

In [8]:
sm.stats.anova_lm(model, typ=3)

,sum_sq,df,F,PR(>F)
Intercept,2.603859,1.0,25659.518916,4.467093e-27
C(label_error),0.003198,1.0,31.509537,3.885228e-05
C(image_noise),0.000351,1.0,3.457214,8.146565e-02
C(labeler),0.000025,1.0,0.249288,6.243721e-01
C(label_error):C(image_noise),0.000053,1.0,0.517778,4.821704e-01
C(label_error):C(labeler),0.001285,1.0,12.659401,2.621796e-03
C(image_noise):C(labeler),0.000280,1.0,2.759665,1.161341e-01
C(label_error):C(image_noise):C(labeler),0.000011,1.0,0.106432,7.484718e-01
Residual,0.001624,16.0,NaN,NaN


##  Now with results for 20 epochs included

In [9]:
root_dir = os.getcwd() + '\\pickles_lower_costs_20_epochs\\' # put all the pickle files to this directory
val_acc = []
avg_val_acc=[]
val_loss=[]
label_error=[]
image_noise=[]
labeler=[]
repetition=[]
for file in os.listdir(root_dir):
    file_name = root_dir + file
    filein = open(file_name, "rb")
    info = pickle.load(filein, encoding="utf8")
    val_acc.extend(info[14][0])
    s_bnn = 0
    for i in range(0, 10):
        s_bnn = s_bnn + val_acc[-5 * i - 1]
    average_valacc = s_bnn / 10
    avg_val_acc.append(average_valacc)
    val_loss.extend(info[16][0])
    label_error.append(re.findall('\d+', file)[9])
    image_noise.append(re.findall('\d+', file)[10])
    labeler.append(re.search(r'True|False', file).group())
    repetition.append(re.findall('\d+', file)[11])

    filein.close()

df1 = pd.DataFrame({'label_error':label_error,
                            'image_noise':image_noise,
                            'labeler':labeler, 
                            'val_acc':avg_val_acc})

In [10]:
df1

,label_error,image_noise,labeler,val_acc
0,0,0,False,0.96874
1,0,0,True,0.95841
2,0,40,False,0.94529
3,0,40,True,0.93818
4,40,0,False,0.93106
5,40,0,True,0.95464
6,40,40,False,0.89981
7,40,40,True,0.94163


In [11]:
rp.summary_cont(df1.groupby(list(df1.columns[:-1].values)))[df1.columns[-1]]

C:\Users\bzi\Miniconda3\envs\myenv\lib\site-packages\scipy\stats\_distn_infrastructure.py:2016: RuntimeWarning: invalid value encountered in greater
  cond0 = self._argcheck(*args) & (scale > 0) & (loc == loc)


N    Mean  SD  SE  95% Conf.  Interval
label_error image_noise labeler                                        
0           0           False    1  0.9687 NaN NaN        NaN       NaN
                        True     1  0.9584 NaN NaN        NaN       NaN
            40          False    1  0.9453 NaN NaN        NaN       NaN
                        True     1  0.9382 NaN NaN        NaN       NaN
40          0           False    1  0.9311 NaN NaN        NaN       NaN
                        True     1  0.9546 NaN NaN        NaN       NaN
            40          False    1  0.8998 NaN NaN        NaN       NaN
                        True     1  0.9416 NaN NaN        NaN       NaN

In [12]:
model_string = "%s ~ "%df1.columns[-1]+"".join(["C(%s)*"%i if i != df1.columns[-1] else '' for i in df1.columns])[:-1]
model_string

'val_acc ~ C(label_error)*C(image_noise)*C(labeler)'

In [13]:
model = ols(model_string, data=df1).fit()

In [14]:
sm.stats.anova_lm(model, typ=3)

C:\Users\bzi\Miniconda3\envs\myenv\lib\site-packages\statsmodels\regression\linear_model.py:1620: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


ValueError: r_matrix performs f_test for using dimensions that are asymptotically non-normal